In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import enum
from pydantic import BaseModel, Field, field_validator
import instructor
from openai import OpenAI
from typing import Literal, Optional, List, Union

In [ ]:
from anthropic import Anthropic

In [17]:
import openai
import instructor

from typing import Iterable, Literal
from pydantic import BaseModel

## OpenAI

In [3]:
import os
from openai import OpenAI

client = OpenAI(
    # This is the default and can be omitted
    api_key=os.environ.get("OPENAI_API_KEY"),
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Say this is a test",
        }
    ],
    model="gpt-3.5-turbo",
)

## Anthropic

In [4]:
import os
from anthropic import Anthropic

client = Anthropic(
    # This is the default and can be omitted
    api_key=os.environ.get("ANTHROPIC_API_KEY"),
    base_url=os.environ.get("ANTHROPIC_API_BASE")
)

message = client.messages.create(
    max_tokens=1024,
    messages=[
        {
            "role": "user",
            "content": "Hello, Claude",
        }
    ],
    model="claude-3-opus-20240229",
)
print(message.content)

[TextBlock(text="Hello! It's nice to meet you. How can I assist you today?", type='text')]


## Instructor

### Minimal examples

In [7]:
class Labels(str, enum.Enum):
    """Enumeration for single-label text classification."""

    SPAM = "spam"
    NOT_SPAM = "not_spam"


class SinglePrediction(BaseModel):
    """
    Class for a single class label prediction.
    """

    class_label: Labels

# Apply the patch to the OpenAI client
# enables response_model keyword
client = instructor.patch(OpenAI())


def classify(data: str) -> SinglePrediction:
    """Perform single-label classification on the input text."""
    return client.chat.completions.create(
        model="gpt-3.5-turbo-0613",
        response_model=SinglePrediction,
        messages=[
            {
                "role": "user",
                "content": f"Classify the following text: {data}",
            },
        ],
    )

# Test single-label classification
prediction = classify("Hello there I'm a Nigerian prince and I want to give you money")
assert prediction.class_label == Labels.SPAM

#### README example

In [5]:
# Define your desired output structure
class UserInfo(BaseModel):
    name: str
    age: int


# Patch the OpenAI client
client = instructor.from_openai(OpenAI())

# Extract structured data from natural language
user_info = client.chat.completions.create(
    model="gpt-3.5-turbo",
    response_model=UserInfo,
    messages=[{"role": "user", "content": "John Doe is 30 years old."}],
)

print(user_info.name)
#> John Doe
print(user_info.age)
#> 30

John Doe
30


#### Anthropic

In [6]:
class User(BaseModel):
    name: str
    age: int


client = instructor.from_anthropic(Anthropic())

# note that client.chat.completions.create will also work
resp = client.messages.create(
    model="claude-3-opus-20240229",
    max_tokens=1024,
    messages=[
        {
            "role": "user",
            "content": "Extract Jason is 25 years old.",
        }
    ],
    response_model=User,
)

assert isinstance(resp, User)
assert resp.name == "Jason"
assert resp.age == 25

#### Felix Vemmer example

In [ ]:
# Patch the OpenAI client
client = instructor.from_openai(OpenAI())

In [ ]:
content = """

In the vast landscape of technological advancements, few innovations hold the promise of revolutionizing our world quite like quantum computing. It’s a realm where traditional binary systems morph into quantum bits or qubits, offering an unparalleled potential to tackle problems deemed insurmountable by classical computers. As we embark on this journey into the enigmatic depths of quantum computing, we’re met with a blend of excitement, curiosity, and a touch of trepidation. At the heart of quantum computing lies the principle of superposition, where qubits can exist in multiple states simultaneously, unlike classical bits, which are limited to either a 0 or 1. This fundamental property enables quantum computers to process vast amounts of information in parallel, promising exponential speed-ups for certain computations. Imagine solving complex optimization problems, simulating molecular structures with precision, or cracking cryptographic codes at an unprecedented pace – these are just glimpses of what quantum computing could offer. However, quantum computing isn’t merely a progression of classical computing; it’s a paradigm shift. Entanglement, another cornerstone of quantum mechanics, adds another layer of complexity. When qubits become entangled, the state of one qubit instantaneously influences the state of another, regardless of the distance between them. This phenomenon opens the door to secure communication channels and novel approaches to information processing. In conclusion, the journey into the enigmatic depths of quantum computing is filled with promise, challenges, and endless possibilities. It’s a journey that beckons us to push the boundaries of our understanding, to embrace uncertainty, and to dare to dream of a future where quantum computers empower humanity to solve the unsolvable. As we navigate this uncharted territory, one thing remains certain – the age of quantum computing has dawned, and the adventure has only just begun."""

In [ ]:
class Metadata(BaseModel):
    meta_title: str = Field(
        ...,
        min_length=50,
        max_length=60,
        description="Meta Title between 50-60 characters. Should be concise, descriptive and include primary keyword.",
    )
    meta_description: str = Field(
        ...,
        min_length=150,
        max_length=160,
        description="Meta Description between 150-160 characters. Should be compelling, summarize the page content and include relevant keywords naturally.",
    )


meta_descrioption = client.chat.completions.create(
    model="gpt-3.5-turbo",
    max_retries=5, # Retry the request 3 times
    response_model=Metadata,
    messages=[
        {"role": "system", "content": content},
    ],
)


In [ ]:
# Here's another example, but with a compound typed field.
class Actor(BaseModel):
    name: str = Field(description="name of an actor")
    country_origin: str = Field(description="Country they were born in")

    @field_validator('country_origin')
    @classmethod
    def country_upper(cls, v: str) -> str:
        if not v.isupper():
            raise ValueError('Must be all caps!')
        return v


actor_query = "Generate the info for a random actor that is not from the US."

actor = client.chat.completions.create(
    model="gpt-3.5-turbo",
    max_retries=2, # Retry the request 3 times
    response_model=Actor,
    messages=[
        {"role": "system", "content": actor_query},
    ],
)
actor

### Trying Page Planner functionality

In [50]:
base = BaseModel
query = "I need a page with a bar and a scatter that filters on the columns 'gdpPerCap` and uses a dropdown as selector. This filter should only apply to the bar chart. The bar chart should be a stacked bar chart, while the scatter chart should be colored by the column 'continent'. I also want a table that shows the data. The title of the page should be `My wonderful jolly dashboard showing a lot of data`."

#### Individual models approach

In [47]:
class AgGridPlanner(base):
    """ Instructions relating to AgGrid or Table components."""
    description: str = Field(..., description="Description of the AgGrid or Table. Include everything that seems to relate to this component.")
    
class GraphPlanner(base):
    """Instructions relating to Charts / Plots / Graphs. Anything we think can be created in plotly."""
    description: str = Field(..., description="Description of the Chart, Plot or Graph. Include everything that seems to relate to this component.")

class CardPlanner(base):
    """Instructions relating to Card /Text. Anything we think should be created in Markdown."""
    description: str = Field(..., description="Description of the Card, Text or Markdown. Include everything that seems to relate to this component.")

In [48]:
class FilterPlanner(base):
    description: str = Field(..., description="Description of the Filter. Include everything that seems to relate to this component.")
    column: Optional[str] = Field(None, description="Column to filter on.")

In [49]:
class PagePlanner(base):
    title: str = Field(..., description="Title of the page. If no description is provided, make a short and concise title from the components.")
    components: List[Union[AgGridPlanner,GraphPlanner,CardPlanner]] #= Field(..., description="List of components of the page and their dedicated description.")
    controls: Optional[List[FilterPlanner]]#Controls

In [ ]:
response_model = PagePlanner

# client = instructor.from_anthropic(Anthropic())
# resp = client.messages.create(
#     model="claude-3-opus-20240229",
#     max_tokens=1024,
#     messages=[
#         {
#             "role": "user",
#             "content": query,
#         }
#     ],
#     response_model=response_model,
# )

client = instructor.from_openai(OpenAI())
resp = client.chat.completions.create(
    model="gpt-4-turbo",
    response_model=response_model,
    messages=[{"role": "system", "content": "You are a world class extractor and your speciality is to extract dashboard components from natural language"},{"role": "user", "content": query}],
)
resp

#### One component, but with a Literal name

In [55]:
comp_type = Literal["AgGrid", "Card", "Graph"]
contr_type = Literal["Filter"]

class Component(BaseModel):
#    """Permissible options to describe components. Needs to be in the format of {"component_name": XXX, "component_description": XXX}."""
    component_name: comp_type
    component_description: str = Field(..., description="Description of the component. Include everything that seems to relate to this component.")
    

class Components(BaseModel):
    components: List[Component]
    
class Control(BaseModel):
#    """Permissible options to describe controls. Needs to be in the format of {"control_name": XXX, "control_description": XXX}."""
    control_name: contr_type
    control_description: str = Field(..., description="Description of the control. Include everything that seems to relate to this control.")

class Controls(BaseModel):
    controls: List[Control]
    
class PagePlanner(BaseModel):
    title: str = Field(..., description="Title of the page. If no description is provided, make a short and concise title from the components.")
    components: Components
    controls: Controls#Optional[List[FilterPlanner]]#

In [56]:
from typing import Iterable
response_model = PagePlanner

# client = instructor.from_anthropic(Anthropic())
# resp = client.messages.create(
#     model="claude-3-opus-20240229",
#     max_tokens=1024,
#     messages=[
#         {
#             "role": "user",
#             "content": query,
#         }
#     ],
#     response_model=response_model,
# )

client = instructor.from_openai(OpenAI())
resp = client.chat.completions.create(
    model="gpt-4-turbo",
    response_model=response_model,
    messages=[{"role": "system", "content": "You are a world class extractor and your speciality is to extract dashboard components from natural language"},{"role": "user", "content": query}],
)
resp

PagePlanner(title='My wonderful jolly dashboard showing a lot of data', components=Components(components=[Component(component_name='Graph', component_description="A stacked bar chart that filters data based on the columns 'gdpPerCap'. The filter is controlled by a dropdown selector. This filter is specifically for this bar chart."), Component(component_name='Graph', component_description="A scatter chart colored by the column 'continent'."), Component(component_name='AgGrid', component_description='A table that shows the data.')]), controls=Controls(controls=[Control(control_name='Filter', control_description="This control is a dropdown that filters the stacked bar chart based on the columns 'gdpPerCap'.")]))

### Blog example

In [32]:
import feedparser
podcast_atom_link = "https://api.substack.com/feed/podcast/1084089.rss" # latent space podcast
parsed = feedparser.parse(podcast_atom_link)
episode = [ep for ep in parsed.entries if ep['title'] == "Why AI Agents Don't Work (yet) - with Kanjun Qiu of Imbue"][0]
episode_summary = episode['summary']
print(episode_summary[:100])

<p><em>Thanks to the </em><em>over 11,000 people</em><em> who joined us for the first AI Engineer Su


In [33]:
from unstructured.partition.html import partition_html
parsed_summary = partition_html(text=''.join(episode_summary)) 
start_of_transcript = [x.text for x in parsed_summary].index("Transcript") + 1
print(f"First line of the transcript: {start_of_transcript}")
text = '\n'.join(t.text for t in parsed_summary[start_of_transcript:])
text = text[:3508] # shortening the transcript for speed & cost

First line of the transcript: 60


In [34]:
text

"Alessio: Hey everyone, welcome to the Latent Space Podcast. This is Alessio, Partner and CTO at Residence at Decibel Partners, and I'm joined by my co-host Swyx, founder of Smol.ai. [00:00:19]\nSwyx: Hey, and today in the studio we have Kanjun from Imbue. Welcome. So you and I have, I guess, crossed paths a number of times. You're formerly named Generally Intelligent and you've just announced your rename, rebrand in huge, humongous ways. So congrats on all of that. And we're here to dive in into deeper detail on Imbue. We like to introduce you on a high level basis, but then have you go into a little bit more of your personal side. So you graduated your BS at MIT and you also spent some time at the MIT Media Lab, one of the most famous, I guess, computer hacking labs in the world. Then you graduated MIT and you went straight into BizOps at Dropbox, where you're eventually chief of staff, which is a pretty interesting role we can dive into later. And then it seems like the founder bug 

In [ ]:
class Person(BaseModel):
    name: str
    school: Optional[str] = Field(..., description="The school this person attended")
    company: Optional[str] = Field(..., description="The company this person works for ")

class People(BaseModel):
    people: List[Person]

In [ ]:
client = instructor.from_openai(OpenAI())

In [ ]:
class Company(BaseModel):
    name:str

class ResearchPaper(BaseModel):
    paper_name:str = Field(..., description="an academic paper reference discussed")
    
class ExtractedInfo(BaseModel):
    people: List[Person]
    companies: List[Company]
    research_papers: Optional[List[ResearchPaper]]

response = client.chat.completions.create(
    model="gpt-4",
    response_model=ExtractedInfo,
    messages=[
        {"role": "user", "content": text},
    ]
)
print(response)

## Pydantic V1

In [3]:
from pydantic.v1 import BaseModel as BaseModelV1

In [4]:
# Define your desired output structure
class UserInfo(BaseModelV1):
    name: str
    age: int


# Patch the OpenAI client
client = instructor.from_openai(OpenAI())

# Extract structured data from natural language
user_info = client.chat.completions.create(
    model="gpt-3.5-turbo",
    response_model=UserInfo,
    messages=[{"role": "user", "content": "John Doe is 30 years old."}],
)

print(user_info.name)
#> John Doe
print(user_info.age)
#> 30

TypeError: UserInfo.__init_subclass__() takes no keyword arguments

#### Parallel function calling

In [19]:
class Weather(BaseModel):
    location: str
    units: Literal["imperial", "metric"]


class GoogleSearch(BaseModel):
    query: str


client = instructor.patch(openai.OpenAI(), mode=instructor.Mode.PARALLEL_TOOLS)  

function_calls = client.chat.completions.create(
    model="gpt-4-turbo-preview",
    messages=[
        {"role": "system", "content": "You must always use tools"},
        {
            "role": "user",
            "content": "What is the weather in toronto and dallas and who won the super bowl?",
        },
    ],
    response_model=Iterable[Union[Weather, GoogleSearch]],  
)

for fc in function_calls:
    print(fc)
    #> location='Toronto' units='metric'
    #> location='Dallas' units='imperial'
    #> query='super bowl winner'

location='Toronto' units='metric'
location='Dallas' units='imperial'
query='Super Bowl winner'


In [21]:
fc

GoogleSearch(query='Super Bowl winner')

#### (Stream) Iterable - for lists of schemas

In [22]:
import instructor
from openai import OpenAI
from typing import Iterable
from pydantic import BaseModel

client = instructor.patch(OpenAI(), mode=instructor.function_calls.Mode.JSON)


class User(BaseModel):
    name: str
    age: int


users = client.chat.completions.create(
    model="gpt-3.5-turbo-1106",
    temperature=0.1,
    response_model=Iterable[User],
    stream=False,
    messages=[
        {
            "role": "user",
            "content": "Consider this data: Jason is 10 and John is 30.\
                         Correctly segment it into entitites\
                        Make sure the JSON is correct",
        },
    ],
)
for user in users:
    print(user)
    #> name='Jason' age=10
    #> name='John' age=30

name='Jason' age=10
name='John' age=30


## Summary

Very slick library that provides a very clear interface with the underlying model

PRO:
- instantiates Pydantic models with ease
- very simple and intuitive API
- support for more complex technologies
- easy support for other model vendors (not sure if this would also hold for marvin)

CON:
- only pydantic V2
- one man show, but a good one!